# RAG using watsonx

## 1. Importing Libraries

In [ ]:
# !pip install ibm-generative-ai

In [ ]:
import os
from dotenv import load_dotenv
import os

from genai.extensions.langchain import LangChainInterface
from genai.schemas import GenerateParams
from genai.credentials import Credentials

from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

## 2. Setting up the API key and URL

In [ ]:
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_url = os.getenv("GENAI_API", None)
creds = Credentials(api_key, api_endpoint=api_url)

## 3. Defining the model and parameters

In [ ]:
model_id = "meta-llama/llama-2-70b-chat"

In [ ]:
params = GenerateParams(
            decoding_method="greedy",
            max_new_tokens=1000,
            min_new_tokens=200,
            temperature=0.7,
        )

## 4. Creating the langchain interface

In [ ]:
llm = LangChainInterface(model=model_id, params=params, credentials=creds)

## 5. Loading the pdf data and storing it in vector database

In [ ]:
def load_pdf(): 
    pdf_name = 'test.pdf'
    loaders = [PyPDFLoader(pdf_name)]

    index = VectorstoreIndexCreator(
        embedding = HuggingFaceEmbeddings(model_name='all-MiniLM-L12-v2'), 
        text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    ).from_loaders(loaders)

    return index

In [ ]:
index = load_pdf()

## 6. Setting a chain using langchain

In [ ]:
chain = RetrievalQA.from_chain_type(llm=llm, 
                                    chain_type='stuff', 
                                    retriever=index.vectorstore.as_retriever(), 
                                    input_key='question')

In [ ]:
response = chain.run("PROMPT OPTIMIZATION EXPERIMENTS results?")

In [ ]:
print(response)